<a href="https://colab.research.google.com/github/Ahmed-M2020/zer0p_notebooks/blob/main/Experiment_7_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from google.colab import drive
import os
import sys
import cv2
from PIL import Image
import copy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
import random
from sklearn.metrics import confusion_matrix
import tqdm
from torch.optim import lr_scheduler
from tqdm import tqdm
# Pytorch
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/filtered_data/final_filtered_df.csv'
df = pd.read_csv(file_path)
# df['Label'] = df['Label'].replace({'good': 1, 'bad': 0})
# df
df.head()
# df.shape

Mounted at /content/drive


,Unnamed: 0,Surface Treatment [bbuchr; msahac],Acc,Resolution [dpi],Rounding [AV],Speed [mm/s],Tep [°C],Head Gap [mm],File,Feature,Label,absolute_path
0,0,bbuchr,150.0,300.0,100.0,50.0,42.0,1,230523_Dataset01_Valeo_Nr01_0.png,dist.1,bad,/content/drive/MyDrive/zerop/FTI Dataset 2023/...
1,1,bbuchr,150.0,300.0,100.0,50.0,42.0,1,230523_Dataset01_Valeo_Nr01_0.png,e.rought1,bad,/content/drive/MyDrive/zerop/FTI Dataset 2023/...
2,2,bbuchr,150.0,300.0,100.0,50.0,42.0,1,230523_Dataset01_Valeo_Nr01_1.png,e.rought2,bad,/content/drive/MyDrive/zerop/FTI Dataset 2023/...
3,3,bbuchr,150.0,300.0,100.0,50.0,42.0,1,230523_Dataset01_Valeo_Nr01_1.png,e.rought3,bad,/content/drive/MyDrive/zerop/FTI Dataset 2023/...
4,4,bbuchr,150.0,300.0,100.0,50.0,42.0,1,230523_Dataset01_Valeo_Nr01_2.png,angle,good,/content/drive/MyDrive/zerop/FTI Dataset 2023/...


In [ ]:
def map_labels(label_string):
    max_labels = 4
    labels = label_string.split(',')
    numerical_labels = [label_map[label] for label in labels if label in label_map]

    # Pad or truncate the list of labels to the fixed length
    if len(numerical_labels) < max_labels:
        numerical_labels += [0] * (max_labels - len(numerical_labels))
    else:
        numerical_labels = numerical_labels[:max_labels]

    return numerical_labels

In [ ]:
df2 = df.groupby('File').agg({
    'Feature': lambda x: ','.join(x),
    'Label': lambda x: ','.join(x),
    'absolute_path': 'first'
}).reset_index()
# Map labels to numerical values
label_map = {'good': 0, 'bad': 1}
# df2['Label'] = df2['Label'].apply(lambda x: label_map[x.split(',')])
df2['Label'] = df2['Label'].apply(map_labels)
df2.head()

,File,Feature,Label,absolute_path
0,20240719_Dataset001_PETSample_01_0.png,dist.1,"[0, 0, 0, 0]",/content/drive/MyDrive/zerop_/FTI Dataset 2024...
1,20240719_Dataset001_PETSample_01_1.png,"e.rought2,e.rought3","[1, 1, 0, 0]",/content/drive/MyDrive/zerop_/FTI Dataset 2024...
2,20240719_Dataset001_PETSample_01_2.png,"e.rought4,dots,dist.6,angle","[0, 0, 0, 0]",/content/drive/MyDrive/zerop_/FTI Dataset 2024...
3,20240719_Dataset001_PETSample_02_0.png,"e.rought1,dist.1","[0, 0, 0, 0]",/content/drive/MyDrive/zerop_/FTI Dataset 2024...
4,20240719_Dataset001_PETSample_02_1.png,"e.rought2,e.rought3","[1, 1, 0, 0]",/content/drive/MyDrive/zerop_/FTI Dataset 2024...


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, features, transform=None, max_labels=4):
        self.image_paths = image_paths
        self.labels = labels
        self.features = features
        self.transform = transform
        self.max_labels = max_labels

        # Check if the lengths match
        if len(self.image_paths) != len(self.labels) or len(self.image_paths) != len(self.features):
            raise ValueError("Length of image_paths, labels, and features must be the same.")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        try:
            img_path = self.image_paths[idx]
            label = self.labels[idx]
            features = self.features[idx]

            image = Image.open(img_path).convert('L')

            if self.transform:
                image = self.transform(image)

            # Pad or truncate labels to fixed length
            # label_padded = label + [0] * (self.max_labels - len(label))
            # label_padded = label_padded[:self.max_labels]

            # Convert to torch tensor
            label = torch.tensor(label).float()
            features = torch.tensor(features).float()

            return image, label, features
        except KeyError:
            print(f"KeyError occurred at index {idx}")
            raise

In [ ]:
torch.manual_seed(42)
np.random.seed(42)

transform_train = transforms.Compose([
    transforms.Resize((256, 256)), #(256, 256)
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=(0, 90)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

transform_valid = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

In [ ]:
from sklearn.model_selection import train_test_split

feature_to_idx = {'dist.1': 0, 'e.rought1': 1, 'e.rought2': 2, 'e.rought3': 3, 'angle': 4, 'dist.6': 5, 'dots': 6, 'e.rought4': 7}
num_features = len(feature_to_idx)

def one_hot_encode_concatenated(features, feature_to_idx, num_features):
    one_hot_encoded = np.zeros((len(features), num_features))
    for i, feature_string in enumerate(features):
        feature_list = feature_string.split(',')
        for feature in feature_list:
            if feature in feature_to_idx:
                one_hot_encoded[i, feature_to_idx[feature]] = 1
            else:
                print(f"Feature '{feature}' not found in label_to_idx")
    return one_hot_encoded

def split_data(df, test_size=0.18, random_state=40):
    # Count the occurrences of each label
    label_counts = df['Label'].value_counts()

    # Identify labels with only one occurrence
    single_occurrence_labels = label_counts[label_counts == 1].index

    if len(single_occurrence_labels) > 0:
        print(f"Warning: Labels {single_occurrence_labels} have only one occurrence each.")
        print("These will be included in the training set.")

        # Separate single occurrence samples
        df_single = df[df['Label'].isin(single_occurrence_labels)]
        df_multiple = df[~df['Label'].isin(single_occurrence_labels)]

        # Split the data with multiple occurrences
        train_df, test_df = train_test_split(df_multiple, test_size=test_size,
                                             random_state=random_state,
                                             stratify=df_multiple['Label'])

        # Add single occurrence samples to training set
        train_df = pd.concat([train_df, df_single])
    else:
        # If all labels have multiple occurrences, proceed with regular stratified split
        train_df, test_df = train_test_split(df, test_size=test_size,
                                             random_state=random_state,
                                             stratify=df['Label'])

    # Prepare features and labels
    X_train = train_df['absolute_path'].tolist()
    X_test = test_df['absolute_path'].tolist()
    y_train = train_df['Label'].tolist()
    y_test = test_df['Label'].tolist()

    # One-hot encode features
    features_train = one_hot_encode_concatenated(train_df['Feature'], feature_to_idx, num_features)
    features_test = one_hot_encode_concatenated(test_df['Feature'], feature_to_idx, num_features)

    return X_train, X_test, y_train, y_test, features_train, features_test

In [ ]:
X_train, X_test, y_train, y_test, features_train, features_test = split_data(df2)

These will be included in the training set.


In [ ]:
len(features_train), len(y_train), len(X_train), len(X_test)

(498, 498, 498, 109)

In [ ]:
features_train[0], features_train[1], features_train[2]

(array([1., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 1., 1., 1.]),
 array([1., 1., 0., 0., 0., 0., 0., 0.]))

In [ ]:
y_train[0], y_train[1], y_train[2], y_train[3]

([0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0])

In [ ]:
feature_to_idx = {'dist.1': 0, 'e.rought1': 1, 'e.rought2': 2, 'e.rought3': 3, 'angle': 4, 'dist.6': 5, 'dots': 6, 'e.rought4': 7}
num_features = len(feature_to_idx)

def one_hot_encode_concatenated(features, feature_to_idx, num_features):
    one_hot_encoded = np.zeros((len(features), num_features))
    for i, feature_string in enumerate(features):
        feature_list = feature_string.split(',')
        for feature in feature_list:
            if feature in feature_to_idx:
                one_hot_encoded[i, feature_to_idx[feature]] = 1
            else:
                print(f"Feature '{feature}' not found in label_to_idx")
    return one_hot_encoded

def calculate_feature_weights(features_train, num_features, device):
    # Count occurrences of each feature
    feature_counts = np.sum(features_train, axis=0)

    # Calculate weights (inverse of frequency)
    weights = 1.0 / (feature_counts + 1e-6)  # adding small epsilon to avoid division by zero

    # Normalize weights
    weights = weights / np.sum(weights)

    # Convert to PyTorch tensor
    feature_weights = torch.FloatTensor(weights).to(device)

    return feature_weights

def calculate_label_weights(y_train, device):

    # y_train_flat = np.array([item for sublist in y_train for item in sublist])  # Flatten the list of lists

    # Count occurrences of each label
    label_counts = np.sum(y_train, axis=0)

    # Calculate weights (inverse of frequency)
    weights = 1.0 / (label_counts + 1e-6)  # adding small epsilon to avoid division by zero

    # Normalize weights
    weights = weights / np.sum(weights)

    # Convert to PyTorch tensor
    label_weights = torch.FloatTensor(weights).to(device)

    return label_weights

# Assuming you have your features_train and y_train from the split_data function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_weights = calculate_feature_weights(features_train, num_features, device)
label_weights = calculate_label_weights(y_train, device)

print(f"Feature Weights: {feature_weights}")
print(f"Label Weights: {label_weights}")

Feature Weights: tensor([0.1199, 0.1310, 0.1271, 0.1216, 0.1261, 0.1341, 0.1158, 0.1243],
       device='cuda:0')
Label Weights: tensor([0.0464, 0.0576, 0.3200, 0.5760], device='cuda:0')


In [ ]:
train_dataset = CustomDataset(X_train, y_train, features_train, transform=transform_train)
test_dataset = CustomDataset(X_test, y_test, features_test, transform=transform_valid)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
train_iterator = iter(train_loader)
data = next(train_iterator)
print(data[0].shape, data[1].shape, data[2].shape)

torch.Size([16, 1, 256, 256]) torch.Size([16, 4]) torch.Size([16, 8])


In [ ]:
class ResNetWithMultiHeads(nn.Module):
    def __init__(self, num_feature_classes, num_label_classes, feature_weights, label_weights):
        super(ResNetWithMultiHeads, self).__init__()

        # Load the pre-trained ResNet50 model
        self.backbone = models.resnet50(pretrained=True)

        # Modify the first convolutional layer to accept 1-channel input
        self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=6, stride=2, padding=3, bias=True)

        # Extract the number of features from the fully connected layer
        num_ftrs = self.backbone.fc.in_features

        # Remove the original fully connected layer (fc)
        self.backbone.fc = nn.Identity()  # Acts as a pass-through layer

        # Shared features processing
        self.shared_fc = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # Multi-label classification heads
        self.feature_head = nn.Linear(256, num_feature_classes)
        self.label_head = nn.Linear(256, num_label_classes)

        # Store the feature and label weights
        self.register_buffer('feature_weights', feature_weights)
        self.register_buffer('label_weights', label_weights)

    def forward(self, image):
        # Forward pass through the backbone
        x = self.backbone(image)

        # Process features
        shared_features = self.shared_fc(x)

        # Output from multi-label classification heads
        feature_output = self.feature_head(shared_features)
        label_output = self.label_head(shared_features)

        return feature_output, label_output

    def weighted_feature_cross_entropy(self, input, target):
        return F.binary_cross_entropy_with_logits(input, target, pos_weight=self.feature_weights)

    def weighted_label_cross_entropy(self, input, target):
        return F.binary_cross_entropy_with_logits(input, target, pos_weight=self.label_weights)

In [ ]:
# Initialize the model and move it to the device
model = ResNetWithMultiHeads(num_feature_classes=8, num_label_classes=4, feature_weights=feature_weights, label_weights=label_weights)
model.to(device)

ResNetWithMultiHeads(
  (backbone): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(6, 6), stride=(2, 2), padding=(3, 3))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
        

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, weight=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.weight = weight

    def forward(self, inputs, targets):
        BCE_loss = F.cross_entropy(inputs, targets, weight=self.weight, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = ((1 - pt) ** self.gamma) * BCE_loss
        return F_loss.mean()

In [ ]:
class WeightedBCELoss(nn.Module):
    def __init__(self, weights=None):
        super(WeightedBCELoss, self).__init__()
        self.weights = weights

    def forward(self, outputs, targets):
        loss = F.binary_cross_entropy(outputs, targets, reduction='none')
        weighted_loss = loss * self.weights
        return weighted_loss.mean()

In [ ]:
def model_train(model, train_loader, valid_loader, device, n_epochs, learning_rate, wd):
    # Loss functions for both heads
    feature_loss_fn = nn.BCEWithLogitsLoss(weight=model.feature_weights)
    label_loss_fn = nn.BCEWithLogitsLoss(weight=model.label_weights)

    optimizer = optim.RAdam(model.parameters(), lr=learning_rate, weight_decay=wd)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # Best validation accuracy tracking
    best_val_acc = -np.inf
    best_weights = None

    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        total_train = 0
        feature_correct_train = 0
        label_correct_train = 0

        with tqdm(enumerate(train_loader), total=len(train_loader), unit="batch") as bar:
            bar.set_description(f"Epoch {epoch + 1}/{n_epochs}")
            for batch_idx, (imgs, label_targets, feature_targets) in bar:
                imgs, feature_targets, label_targets = imgs.to(device), feature_targets.to(device), label_targets.to(device)
                optimizer.zero_grad()

                # Forward pass
                feature_outputs, label_outputs = model(imgs)

                # Compute the losses
                feature_loss = model.weighted_feature_cross_entropy(feature_outputs, feature_targets)
                label_loss = model.weighted_label_cross_entropy(label_outputs, label_targets)
                loss = feature_loss + label_loss

                # Backpropagation and optimization
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

                # print(f"Feature_target_shape:{feature_targets}\n Feature_output_shape {feature_outputs}")
                # print(f"Label_target_shape:{label_targets}\n Label_output_shape {label_outputs}")

                # Accuracy for feature and label heads
                # feature_correct_train += ((feature_outputs > 0) == feature_targets).float().sum().item()
                # label_correct_train += ((label_outputs > 0) == label_targets).float().sum().item()
                # total_train += feature_targets.numel() + label_targets.numel()

                # # Training accuracies
                # train_acc_feature = feature_correct_train / (total_train / 2)
                # train_acc_label = label_correct_train / (total_train / 2)

                feature_correct_train += ((feature_outputs > 0.5) == feature_targets).sum().item()
                label_correct_train += ((label_outputs > 0.5) == label_targets).sum().item()
                total_train += feature_targets.size(0)

                # Training accuracies
                train_acc_feature = feature_correct_train / (total_train * feature_targets.size(1))
                train_acc_label = label_correct_train / (total_train * label_targets.size(1))

                bar.set_postfix(
                    loss=running_loss / (batch_idx + 1),
                    feature_acc=train_acc_feature,
                    label_acc=train_acc_label
                )

        scheduler.step()

        # Validation
        model.eval()
        feature_correct_val = 0
        label_correct_val = 0
        total_val = 0

        with torch.no_grad():
            for imgs, label_targets, feature_targets in valid_loader:
                imgs, feature_targets, label_targets = imgs.to(device), feature_targets.to(device), label_targets.to(device)

                # Forward pass
                feature_outputs, label_outputs = model(imgs)

                # Accuracy for feature and label heads
        #         feature_correct_val += ((feature_outputs > 0) == feature_targets).float().sum().item()
        #         label_correct_val += ((label_outputs > 0) == label_targets).float().sum().item()
        #         total_val += feature_targets.numel() + label_targets.numel()

        # val_acc_feature = feature_correct_val / (total_val / 2)
        # val_acc_label = label_correct_val / (total_val / 2)
        # avg_val_acc = (val_acc_feature + val_acc_label) / 2
                # Accuracy for feature and label heads
                feature_correct_val += ((feature_outputs > 0.5) == feature_targets).sum().item()
                label_correct_val += ((label_outputs > 0.5) == label_targets).sum().item()
                total_val += feature_targets.size(0)

        val_acc_feature = feature_correct_val / (total_val * feature_targets.size(1))
        val_acc_label = label_correct_val / (total_val * label_targets.size(1))
        avg_val_acc = (val_acc_feature + val_acc_label) / 2

        print(f"Epoch {epoch + 1} Validation Accuracy - Feature: {val_acc_feature * 100:.2f}%, Label: {val_acc_label * 100:.2f}%, Average: {avg_val_acc * 100:.2f}%")

        # Save the best model weights
        if avg_val_acc > best_val_acc:
            best_val_acc = avg_val_acc
            best_weights = copy.deepcopy(model.state_dict())
            print(f"Best validation accuracy ({best_val_acc * 100:.2f}%)")

    return best_val_acc, best_weights

In [ ]:
best_acc, best_weights = model_train(model, train_loader, valid_loader, device, n_epochs=20, learning_rate=1e-4, wd= 1e-5)
if best_acc > 0.895:
    accuracy_str = f'{best_acc * 100:.2f}'.replace('.', '_')
    save_path = f'/content/drive/MyDrive/Results/saved_models/resnet18/filtered/best_model_{accuracy_str}.pth'
    torch.save(best_weights , save_path)
    # model.load_state_dict(best_weights)
    print(f"Saved the Best Accuracy: {best_acc * 100:.2f}%")

Epoch 1/20: 100%|██████████| 32/32 [00:42<00:00,  1.33s/batch, feature_acc=0.726, label_acc=0.81, loss=0.429]


Epoch 1 Validation Accuracy - Feature: 73.05%, Label: 81.65%, Average: 77.35%
Best validation accuracy (77.35%)


Epoch 2/20: 100%|██████████| 32/32 [00:43<00:00,  1.37s/batch, feature_acc=0.725, label_acc=0.81, loss=0.348]


Epoch 2 Validation Accuracy - Feature: 73.05%, Label: 81.65%, Average: 77.35%


Epoch 3/20: 100%|██████████| 32/32 [00:42<00:00,  1.34s/batch, feature_acc=0.726, label_acc=0.81, loss=0.271]


Epoch 3 Validation Accuracy - Feature: 73.05%, Label: 81.65%, Average: 77.35%


Epoch 4/20:  12%|█▎        | 4/32 [00:05<00:40,  1.44s/batch, feature_acc=0.725, label_acc=0.832, loss=0.251]